In [ ]:
#!/bin/python
import glob
import pandas as pd
import numpy as np
import os
# import cPickle
import pickle
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster.k_means_ import KMeans
import sys
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
import re
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
from lightgbm import LGBMClassifier
import sys
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
sys.argv = ['scripts/create_kmeans.py', './kmeans', 100, './surf']

In [ ]:
kmeans_model = sys.argv[1]+'/knn_{}_model.sav'.format(sys.argv[2])
cluster_num = int(sys.argv[2])
surf_file_list = sys.argv[3]

In [ ]:
# #Usage: {0} kmeans_model, cluster_num, file_list
# model_path = 'knn_100_model'
# n_clusters = 100
# video_list = './list/train.video'

# kmeans_model = './kmeans/'+model_path+'.sav'; file_list = video_list
# cluster_num = int(n_clusters)

In [ ]:
path = os.path.join(surf_file_list, '*.pkl')

filelist = []

for file in glob.glob(path):
    filelist.append(file)

In [ ]:
def get_features(k, model, path_list, n_frames = 500):
    loaded_model = model
    k = k
    start_time = time.time()
#     features_dict = dict()
    features_list = []
    filelist = path_list
    except_list = ['./surf/HVC5510.pkl', './surf/HVC721.pkl', './surf/HVC3124.pkl']
#     for i in range(len(filelist)):
    for i in range(100):
        if filelist[i] in except_list:
            array_data = np.zeros((1, 64))
        else:
            with open(filelist[i], 'rb') as f:
                    try:
                        data = pickle.load(f)
                        if len(data) != 0:
                            foo_list = []
                            if len(data) >= 500:                            
                                rsr_idx = np.random.choice(len(data), n_frames)
                                for idx in rsr_idx:
                                    if len(data[idx].shape) == 3:
                                        foo_list.append(data[idx][0])
                                    else:
                                        pass
                            else:
                                for idx in range(len(data)):
                                    if len(data[idx].shape) == 3:
                                        foo_list.append(data[idx][0])
                                    else:
                                        pass
                        else:
                            print('{}th Pickle is Empty!! Skip it!!'.format(i))                    
                        array_data = np.concatenate(foo_list, axis = 0)
                    except EOFError:
        #                     array_data = np.zeros((30000, 64))
                        array_data = np.zeros((1, 64))

        pred_centers = loaded_model.predict(array_data)
        num_clusters = k
        bow_preds = np.zeros((1, num_clusters))

        for ind in pred_centers:
            bow_preds[0, ind] += 1
        norm_feat = (1.0 * bow_preds)/np.sum(bow_preds)
#         features_dict[i] = norm_feat
        features_list.append(norm_feat)
#         print(i)
        if i % 100 == 0:
            print('{}th step is progressing!!!'.format(i))
        else: pass
    print("===== The time consuming of getting features : {} seconds =====".format((time.time() - start_time)))    
    return features_list

In [ ]:
#foo = get_features(cluster_num, model, filelist, n_frames = 100)
# data_array = np.concatenate(foo, axis = 0)
# data_mart = pd.DataFrame(data_array)
# data_mart

In [ ]:
# # with open(filelist[0], 'rb') as f:
# #     data = pickle.load(f)
foo = get_features(100, model, filelist, n_frames = 100)
data_array = np.concatenate(foo, axis = 0)
data_mart = pd.DataFrame(data_array)
data_mart.to_csv('./total_surf_features_k{}.csv'.format(cluster_num))

In [ ]:
# load the kmeans model
# kmeans_model = './{}.sav'.format(sys.argv[1])
model = pickle.load(open(kmeans_model,"rb"))

In [ ]:
total_features = pd.read_csv('./total_surf_features_k{}.csv'.format(cluster_num))
total_features.head(3)

In [ ]:
video_name_ind = []
for i in range(len(filelist)):
    match_front = re.search('surf/', filelist[i])
    match_end = re.search('.pkl', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})  

In [ ]:
k = cluster_num
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

In [ ]:
total_data = pd.concat([video_name, total_features], axis = 1)
total_data.columns = column_name
train_ind = pd.read_csv('./list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./list/test.video', sep = ' ', header = None)

In [ ]:
train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

In [ ]:
train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)

In [ ]:
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

In [ ]:
total_mart = total_data.merge(data_lable, how = 'right', on = 'video')
total_mart = total_mart.fillna(0)

train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

total_mart.to_csv('./datamart_surf_total_k{}.csv'.format(cluster_num), index=False)
train_mart.to_csv('./datamart_surf_train_k{}.csv'.format(cluster_num), index=False)
valid_mart.to_csv('./datamart_surf_valid_k{}.csv'.format(cluster_num), index=False)
test_mart.to_csv('./datamart_surf_test_k{}.csv'.format(cluster_num), index=False)

In [ ]:
# Generate k-means features for videos; each video is represented by a single vector

if __name__ == '__main__':
    if len(sys.argv) != 4:
        print "Usage: {0} kmeans_model, cluster_num, file_list".format(sys.argv[0])
        print "kmeans_model -- path to the kmeans model"
        print "cluster_num -- number of cluster"
        print "file_list -- the list of videos"
        exit(1)
    warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
    kmeans_model = './'+sys.argv[1]+'.sav'; file_list = sys.argv[3]
    cluster_num = int(sys.argv[2])

    # load the kmeans model
    # kmeans_model = './{}.sav'.format(sys.argv[1])
    kmeans = pickle.load(open(kmeans_model,"rb"))
    model = kmeans
    
    # path = './hw1_git/11775-hws/videos/*.mp4'
    path = './mfcc/*.csv'

    filelist = []

    for file in glob.glob(path):
        filelist.append(file)
        
    def get_features(k, model, path_list):
        loaded_model= model
        start_time = time.time()
        features_dict = dict()
        filelist = path_list
        for i in range(len(filelist)):
    #     for i in range(10):        
            if i % 1000 == 0: 
                print('{}th step progressing....'.format(i)) 
            else: 
                pass
            data = pd.read_csv(filelist[i], sep = ';', header = None)
            pred_centers = loaded_model.predict(data)
            num_clusters = k
            bow_preds = np.zeros((1, num_clusters))

            for ind in pred_centers:
                bow_preds[0, ind] += 1
            norm_feat = (1.0 * bow_preds)/np.sum(bow_preds)
            features_dict[i] = pd.DataFrame(norm_feat)

        features_total = features_dict[0].copy()
        for i in range(1, len(features_dict)):
            foo = features_dict[i].copy()
            features_total = pd.concat([features_total, foo], axis = 0)
            features_total = features_total.reset_index().drop('index', axis = 1)

        print("===== The time consuming of getting features : {} seconds =====".format((time.time() - start_time)))
        return features_total  
    
    total_features = get_features(k = cluster_num, model = model, path_list = filelist)    
    total_features.to_csv('./total_features_k{}.csv'.format(cluster_num), index=False)
    
    total_features = pd.read_csv('./total_features_k{}.csv'.format(cluster_num))
    total_features.head(3)

    video_name_ind = []
    for i in range(len(filelist)):
        match_front = re.search('mfcc/', filelist[i])
        match_end = re.search('.mfcc.csv', filelist[i])
        video_name_ind.append(filelist[i][match_front.end():match_end.start()])
        video_name = pd.DataFrame({'video': video_name_ind})    
    
    # Making features columns
    k = cluster_num
    column_name = ['video']
    for i in range(k):
        column_name.append('feature_{}'.format(i))

    total_data = pd.concat([video_name, total_features], axis = 1)
    total_data.columns = column_name

    train_ind = pd.read_csv('./list/train', sep = ' ', header = None)
    valid_ind = pd.read_csv('./list/val', sep = ' ', header = None)
    test_ind = pd.read_csv('./list/test.video', sep = ' ', header = None)

    train_ind['Data'] = 'TRAIN'
    valid_ind['Data'] = 'VALID'
    test_ind[1] = 'UNK'
    test_ind['Data'] = 'TEST'

    train_ind.columns = ['video','target','Data']
    valid_ind.columns = ['video','target','Data']
    test_ind.columns = ['video','target','Data']

    data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)
    # data_lable['target_p001'] = 
    data_lable['target_p001'] = data_lable['target']
    data_lable['target_p002'] = data_lable['target']
    data_lable['target_p003'] = data_lable['target']
    data_lable['target_p001_10'] = 1
    data_lable['target_p002_10'] = 1
    data_lable['target_p003_10'] = 1

    data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
    data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
    data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
    data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
    data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
    data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

    total_mart = total_data.merge(data_lable, how = 'right', on = 'video')
    total_mart = total_mart.fillna(0)
    
    train_mart = total_mart[total_mart['Data'] == 'TRAIN']
    valid_mart = total_mart[total_mart['Data'] == 'VALID']
    test_mart  = total_mart[total_mart['Data'] == 'TEST']
    
    total_mart.to_csv('./datamart_total_k{}.csv'.format(cluster_num), index=False)
    train_mart.to_csv('./datamart_train_k{}.csv'.format(cluster_num), index=False)
    valid_mart.to_csv('./datamart_valid_k{}.csv'.format(cluster_num), index=False)
    test_mart.to_csv('./datamart_test_k{}.csv'.format(cluster_num), index=False)
    print "K-means features generated successfully!"


In [ ]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_lgbm(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = LGBMClassifier(random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of LightGBM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

In [ ]:
sys.argv = ['scripts/train_model.py', 'P001', 100, './surf_model', 'surf']
event_name = sys.argv[1]
feat_dim = sys.argv[2]
output_file = sys.argv[3]
model_type = sys.argv[4]

In [ ]:
train_mart = pd.read_csv('./datamart_{}_train_k{}.csv'.format(model_type, feat_dim))
valid_mart = pd.read_csv('./datamart_{}_valid_k{}.csv'.format(model_type, feat_dim))
test_mart  = pd.read_csv('./datamart_{}_test_k{}.csv'.format(model_type, feat_dim))

In [ ]:
#Choose for each model based on MAP
Xgb_results_p001, _, surf_model_p001 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002, _, surf_model_p002 = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003, _, surf_model_p003 = modeling_ap_lgbm(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
filename = os.path.join(output_file, event_name+'_'+model_type+'.model.sav')
filename

In [ ]:
pickle.dump(surf_model_p001, open(filename, 'wb'))

In [ ]:
#python2 scripts/test_svm.py mfcc_pred/$event.model "kmeans/" $feat_dim_mfcc mfcc_pred/${event}_mfcc.lst || exit 1;
# sys.argv = ['scripts/test_model.py', 'P001', 'cnn_model', 1280, 'cnn_pred/P001_cnn.list', 'cnn']
sys.argv = ['scripts/test_model.py', 'P001', 'surf_model', 100, 'surf']

### Test_model.py

In [ ]:
if __name__ == '__main__':
    if len(sys.argv) != 4:
        print("Usage: {0} model_file feat_dir feat_dim output_file".format(sys.argv[0]))
        print("model_file -- path of the trained svm file")
        print("feat_dir -- dir of feature files")
        print("output_file -- path to save the prediction score")
        print("model_type -- mfcc or asrs")
        exit(1)

    event_name = sys.argv[1]
    model_file = sys.argv[2]
    feat_dim = int(sys.argv[3])
    model_type = sys.argv[4]
    print(event_name)
    print(model_file)
    print(feat_dim)
    print(model_type)

In [ ]:
filename = os.path.join(model_file, event_name+'_'+model_type+'.model.sav')
model = pickle.load(open(filename, 'rb'))
test_mart  = pd.read_csv('./datamart_surf_test_k{}.csv'.format(feat_dim))
X_test = test_mart.iloc[:, 1:feat_dim + 1]
scores = model.predict_proba(X_test)
scores_df = pd.DataFrame(scores)
scores_df.columns = ['N', 'Y']
scores_total = pd.concat([test_mart, scores_df], axis = 1)
# scores_total.head(3)

test_list = pd.read_csv('./list/test.video', header = None)
test_list.columns = ['video']

output_file = 'surf_pred/'+event_name+'_'+model_type+'.list'
final_list = test_list.merge(scores_total, how = 'left', on = 'video')[['video', 'Y']]
# final_list.to_csv(output_file+'.video'+'.csv', index = False)
# final_list['Y'].to_csv(output_file+'.csv', index = False, header = True)
final_list.to_csv(output_file+'.video'+'.txt', index = False, header = False)
final_list['Y'].to_csv(output_file+'.txt', index = False, header = False)

In [ ]:
SVM_results_p001 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003 = modeling_ap_SVM(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
AdaB_results_p001, _ = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002, _ = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003, _ = modeling_ap_AdaB(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Boost_results_p001, _ = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002, _ = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003, _ = modeling_ap_Boost(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Xgb_results_p001, _ = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002, _ = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003, _ = modeling_ap_xgb(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
LGBM_results_p001, _ = modeling_ap_lgbm(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
LGBM_results_p002, _ = modeling_ap_lgbm(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003, _ = modeling_ap_lgbm(k=100, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')